# Final Project Report: Group X

Technical report of the Final Project of the course Advanced Business Analytics

### Introduction
Our project revolves around the use of advanced analytics and Natural Lenguage Processing tools for the recommendation of products close to expiration in supermarkets of the Sailing Group. By accessing the data available on the different APIs offered by the Sailing group, we set to focus on the real-time data of discounted products in order to provide tailored recommendation for the user based on data and feedback the user provides. Initially, we explore simple recommendation techniques based on quantitative metrics such as discounted price and we proceed to create a system that recommends products based on the user's specific wish. Finally, we focus on LLMs and how via prompt engineering they can offer a new prespective and window of opportunity for recommender systems, not only via single-item recommendation but also via generative approaches. 

### Motivation
- Here we explin a bit of the motivation on why food waste and why using LLMs are a need promising alternative

### Data analysis and visualization
- Initial call on the API
- Retrieval of as much data from different stores as possible. 
- Provide stats on this data (Thor's visualization notebook)
- Conclude explining challenges and possibilities (challenges: lots of already-made food, hard to categorize items and short selection of certain products like veggies)(possibilities: set step for further sections)

### Simple recommendation techniques
- Andres's notebook on recommendation based on discount

### Word embedding for user-guided recommendation 
- Anika's clustering for 
- Thor's notebook for looking up similar products to a given word


### LLMs and prompt engineering for advanced recommendation
- Explain challenges and limitations of previous recommendation methods
- Introduce recipe generation and user engagement ideas
- Oriol's notebook


While our earlier techniques successfully incorporated basic heuristics and word embeddings to support user-guided recommendations, these methods had several key limitations. Most notably, they required a predefined logic for mapping products and relied heavily on manual feature crafting or keyword matching. This proved inadequate when addressing ambiguous or open-ended user requests, such as “I want to cook a healthy vegetarian dinner” or “give me a lunch idea with what’s about to expire.”

To overcome these challenges, we turned to large language models (LLMs) and prompt engineering. LLMs, with their capacity to understand and generate human-like text, enabled a more flexible and context-aware recommendation experience. Rather than matching based on word similarity, LLMs allowed us to interpret the user’s intent, understand constraints (e.g., dietary preferences, budget), and even generate novel recipe suggestions using the discounted products currently available in nearby stores.

We designed prompts that incorporate user preferences, product metadata, and store availability data into a unified query. These prompts were dynamically constructed using Python and passed to the LLM via API calls. In practice, this allowed us to achieve two main innovations:

Conversational Recommendation: Users could input free-text descriptions of their needs (e.g., “I have two kids and want something quick for dinner”), and the LLM would generate relevant product bundles or meals, incorporating discounted items retrieved via the Sailing Group APIs.

Generative Recipes: By combining the list of available ingredients with user preferences, we generated simple recipes or meal ideas. For instance, if a user had yogurt, granola, and near-expiring berries, the model could suggest a breakfast parfait, along with preparation steps and nutritional insights.

Despite their promise, LLMs also posed challenges:

Hallucinations: Occasionally, the models recommended non-existent or unavailable items.

API cost and latency: Each LLM call added overhead to system responsiveness, which would require optimization for real-time deployment.

Lack of domain knowledge: Without fine-tuning, LLMs sometimes produced unrealistic combinations (e.g., pairing incompatible flavors or suggesting cooked versions of ready-to-eat meals).

Nevertheless, our experiments clearly indicate that LLMs provide a powerful, human-centric layer for recommendation systems. They are particularly useful in contexts where creativity, adaptability, and user interaction are valued. Combined with structured data and traditional filtering mechanisms, LLMs can enhance both user engagement and product utilization—ultimately contributing to the broader goal of reducing food waste.

To push beyond rule-based and embedding-based recommendations, we implemented an adaptive system leveraging OpenAI's large language models (LLMs) for real-time, personalized recipe generation. This system, called the AdaptiveRecipeAgent, is a feedback-aware, prompt-driven recipe generator that responds dynamically to a user’s ingredient availability and expressed preferences.

The design centers around an evolving user profile (memory), which collects feedback across interactions and integrates it into future prompt construction. The memory stores banned ingredients, dietary restrictions, and preferences such as desired cuisine, dish type, and flavor profiles.

Key Features:

- Contextual Memory: Tracks banned ingredients and feedback-based preferences across multiple user inputs.

- Dynamic Prompt Building: Uses real-time memory and available ingredients to construct effective prompts.

- Conversational Adaptation: Can adapt recipes over multiple feedback cycles, making the experience more interactive and personalized.

- Below is the core implementation, annotated for technical clarity:

In [ ]:
import openai
from openai import OpenAI

# Initialize the OpenAI client (you can also use openai.api_key = "..." style if preferred)
client = OpenAI(api_key="your-api-key")  # Replace with your actual key or use environment variable

class AdaptiveRecipeAgent:
    def __init__(self, ingredients):
        self.ingredients = set(ingredients)
        self.memory = {
            "feedback_log": [],
            "banned_ingredients": set(),
            "preferences": [],
            "dietary_restrictions": [],
            "cuisine": None,
            "dish_type": None
        }

    def update_from_feedback(self, feedback):
        self.memory["feedback_log"].append(feedback)
        feedback_lower = feedback.lower()

        for ingredient in self.ingredients:
            if f"don't use {ingredient.lower()}" in feedback_lower or f"no {ingredient.lower()}" in feedback_lower:
                self.memory["banned_ingredients"].add(ingredient)

        if "simpler" in feedback_lower or "easy" in feedback_lower:
            self.memory["preferences"].append("simpler recipe")

        if "spicy" in feedback_lower:
            self.memory["preferences"].append("make it spicy")

        if "dairy-free" in feedback_lower:
            self.memory["dietary_restrictions"].append("dairy-free")

        if "gluten-free" in feedback_lower:
            self.memory["dietary_restrictions"].append("gluten-free")

        self.memory["preferences"].append(feedback.strip())

    def build_prompt(self):
        prompt = "You are a creative chef. Generate a recipe based on the following:\n\n"
        prompt += f"Available ingredients: {', '.join(self.ingredients - self.memory['banned_ingredients'])}\n"

        if self.memory["dietary_restrictions"]:
            prompt += f"Dietary restrictions: {', '.join(set(self.memory['dietary_restrictions']))}\n"

        if self.memory["preferences"]:
            prompt += f"User preferences: {', '.join(set(self.memory['preferences']))}\n"

        if self.memory["cuisine"]:
            prompt += f"Cuisine: {self.memory['cuisine']}\n"

        if self.memory["dish_type"]:
            prompt += f"Dish type: {self.memory['dish_type']}\n"

        prompt += "\nPlease include:\n- A recipe title\n- List of ingredients\n- Step-by-step instructions"
        return prompt

    def generate_recipe(self, model="gpt-4"):
        prompt = self.build_prompt()

        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are an expert recipe generator that adapts based on user feedback."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=700
        )

        return response.choices[0].message.content

In [2]:
agent = AdaptiveRecipeAgent(["chicken", "rice", "broccoli", "cheese", "milk"])

print(agent.generate_recipe())

agent.update_from_feedback("Don't use cheese, I want it dairy-free and simpler.")
print(agent.generate_recipe())

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
